Datashader is designed to make it simple to work with even very large
datasets. To get good performance, it is essential that each step in the
overall processing pipeline be set up appropriately. Below we share some
of our suggestions based on our own [benchmarking](https://github.com/holoviz/datashader/issues/313) and optimization
experience, which should help you obtain suitable performance in your
own work.

## File formats

Based on our [testing with various file formats](https://github.com/holoviz/datashader/issues/129), we recommend storing
any large columnar datasets in the [Apache Parquet](https://parquet.apache.org/) format when
possible, using the [fastparquet](https://github.com/dask/fastparquet) library with [Snappy](https://github.com/andrix/python-snappy) compression:

```
>>> import dask.dataframe as dd
>>> dd.to_parquet(filename, df, compression="SNAPPY")
```

If your data includes categorical values that take on a limited, fixed
number of possible values (e.g. "Male", "Female"),
Parquet's categorical columns use a more memory-efficient data representation and
are optimized for common operations such as sorting and finding uniques.
Before saving, just convert the column as follows:

```
>>> df[colname] = df[colname].astype('category')
```

By default, numerical datasets typically use 64-bit floats, but many
applications do not require 64-bit precision when aggregating over a
very large number of datapoints to show a distribution. Using 32-bit
floats reduces storage and memory requirements in half, and also
typically greatly speeds up computations because only half as much data
needs to be accessed in memory. If applicable to your particular
situation, just convert the data type before generating the file:

```
>>> df[colname] = df[colname].astype(numpy.float32)
```

## Data libraries

Datashader performance will vary significantly depending on the library used to represent the data in Python, because different libraries have very different abilities to use the available processing power and memory. Moreover, different libraries are appropriate for different types of data, due to how those libraries organize and store the data internally as well as the operations they provide for working with the data. The data libraries currently supported by Datashader are:

- [Pandas](https://pandas.pydata.org): Basic columnar data support, typically lower performance than the other options where those are supported. Does not typically support multi-threaded operation that can make full use of your CPU's cores, and is generally limited to datasets that fit in your CPU's accessible memory. Does not support ragged arrays like polygons efficiently on its own.
- [Dask](https://dask.org): Multi-threaded support built on Pandas that can make full use of your CPU's cores, distributed support for using multiple CPUs in a cluster, HPC system or the cloud, and out-of-core support for datasets larger than memory. 
- [cuDF](https://github.com/rapidsai/cudf): NVIDIA GPU support, for datasets in GPU memory and processed by a GPU.
- [Dask_cuDF](https://rapidsai.github.io/projects/cudf/en/0.10.0/10min.html): Multi-GPU support for computation distributed across multiple NVIDIA GPUs on the same or different machines
- [SpatialPandas](https://github.com/holoviz/spatialpandas): Pandas extended to support efficient storage and computation on ragged arrays for polygons and variable-length lines, typically on one core of one CPU
- [Dask+SpatialPandas](https://github.com/holoviz/spatialpandas): SpatialPandas combined with Dask to support multi-core, distributed, and out-of-core processing.
- [Xarray](http://xarray.pydata.org): Efficient, Dask-based multidimensional (not columnar or ragged) array operation, including support for multi-core, distributed, and out-of-core processing on CPUs.
- [Xarray+CuPy](https://cupy.chainer.org): Xarray multidimensional array processing on an NVIDIA GPU.

Datashader's current release supports these libraries for nearly all of the Canvas glyph types (points, lines, etc.) where they would apply. Supported combinations of glyph and data library are listed in this table, where the entries mean:

- **Yes**: Supported
- **No**: Not (yet) supported, but could be with sufficient development effort (feel free to contribute effort or funding!)
- **-**: Not supported because that combination is not normally appropriate or useful (e.g. columnar data libraries will not provide efficient multidimensional array support)
- **(Date)**: Not supported in the current release, but currently being investigated and implemented and expected to be released by the indicated date.

<style type="text/css">.arbit .trary a { color: inherit; }.arbit .trary
.sL{text-align:center;padding:2px 2px 2px 2px;background-color:#ffffff;font-weight:bold;width:80px}.arbit .trary
.sG{text-align:center;padding:2px 2px 2px 2px;background-color:#ffffff;font-weight:bold;font-family:monospace}.arbit .trary
.sY{text-align:center;padding:2px 2px 2px 2px;background-color:#b7e1cd;}.arbit .trary 
.sN{text-align:center;padding:2px 2px 2px 2px;background-color:#f4c7c3;}.arbit .trary
.sM{text-align:center;padding:2px 2px 2px 2px;background-color:#fce8b2;}.arbit .trary
</style>

<div class="arbit">
<table class="trary" cellspacing="0" cellpadding="0">
<thead><tr>

<th></th>
<th class="sL">Glyph</th>
<th class="sL">Pandas</th>
<th class="sL">Dask+Pandas</th>
<th class="sL">cuDF</th>
<th class="sL">Dask_cuDF</th>
<th class="sL">SpatialPandas</th>
<th class="sL">Dask+SpatialPandas</th>
<th class="sL">Xarray</th>
<th class="sL">Xarray+CuPy</th>
</tr></thead><tbody>

<tr><th></th>
<td class="sG">Canvas.points</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sN">No</td>
<td class="sN">No</td></tr>

<tr><th></th>
<td class="sG">Canvas.line</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sN">No</td>
<td class="sN">No</td></tr>

<tr><th></th>
<td class="sG">Canvas.area</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sN">No</td>
<td class="sN">No</td></tr>

<tr><th></th>
<td class="sG">Canvas.trimesh</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sN">No</td>
<td class="sN">No</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sN">No</td>
<td class="sN">No</td></tr>

<tr><th></th>
<td class="sG">Canvas.raster</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sY">Yes</td>
<td class="sM">2/2020</td></tr>

<tr><th></th>
<td class="sG">Canvas.quadmesh</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sY">Yes</td>
<td class="sM">2/2020</td></tr>

<tr><th></th>
<td class="sG">Canvas.polygons</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sM">-</td>
<td class="sY">Yes</td>
<td class="sY">Yes</td>
<td class="sM">-</td>
<td class="sM">-</td></tr>

</tbody></table></div>

In general, all it takes to use the indicated data library for a particular glyph type is to instantiate a DataFrame (Pandas, Dask, cuPy, SpatialPandas) or DataArray/DataSet (Xarray), and then pass it to the appropriate `ds.Canvas` method call, as illustrated in the various examples in the user guide and topics.

## Using Dask efficiently

Even on a single machine, a Dask DataFrame
typically give higher performance than Pandas, because it
makes good use of all available cores, and it also supports out-of-core
operation for datasets larger than memory.

Dasks works on chunks of the data at any one time, called partitions.
With Dask on a single machine, a rule of thumb for the number of
partitions to use is `multiprocessing.cpu_count()`, which allows Dask to
use one thread per core for parallelizing computations.

When the entire dataset fits into memory at once, you can (and should) persist the
data as a Dask dataframe prior to passing it into datashader, to ensure
that data only needs to be loaded once:

```
>>> from dask import dataframe as dd
>>> import multiprocessing as mp
>>> dask_df = dd.from_pandas(df, npartitions=mp.cpu_count())
>>> dask_df.persist()
...
>>> cvs = datashader.Canvas(...)
>>> agg = cvs.points(dask_df, ...)
```